In [1]:
import os
import time

import cv2
import matplotlib.pyplot as plt
import numpy as np
import png
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from colormap.colors import Color, hex2rgb
from sklearn.metrics import average_precision_score as ap_score
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from tqdm import tqdm
import torch.nn.functional as F

from dataset import FacadeDataset

from train import Net
from train import train, test, get_result


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# TODO change data_range to include all train/evaluation/test data.
# TODO adjust batch_size.
train_data = FacadeDataset(flag='train', data_range=(0,800), onehot=False)
train_loader = DataLoader(train_data, batch_size=10)
val_data = FacadeDataset(flag='train', data_range=(800,906), onehot=False)
val_loader = DataLoader(val_data, batch_size=10)
test_data = FacadeDataset(flag='test_dev', data_range=(0,114), onehot=False)
test_loader = DataLoader(test_data, batch_size=5)

load train dataset start
    from: ./starter_set/
    range: [0, 800)
load dataset done
load train dataset start
    from: ./starter_set/
    range: [800, 906)
load dataset done
load test_dev dataset start
    from: ./starter_set/
    range: [0, 114)
load dataset done


In [7]:
N_CLASS=5
#torch.nn.ConvTranspose2d

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.n_class = N_CLASS
        self.layers = nn.Sequential(
            #########################################
            ###        TODO: Add more layers      ###
            #########################################
            nn.Conv2d(3, self.n_class, 1, padding=0),
            nn.ReLU(inplace=True)
        )
        self.encoder = nn.Sequential(
        	# conv layer 1
        	nn.Conv2d(3, 64, 3, padding=1),
        	nn.ReLU(inplace=True),
        	nn.Conv2d(64, 64, 3, padding=1),
        	nn.ReLU(inplace=True),
        	nn.MaxPool2d(2, stride=2, ceil_mode=True),
        	# conv layer 2
        	nn.Conv2d(64, 128, 3, padding=1),
        	nn.ReLU(inplace=True),
        	nn.Conv2d(128, 128, 3, padding=1),
        	nn.ReLU(inplace=True),
        	nn.MaxPool2d(2, stride=2, ceil_mode=True)
        	)
        self.decoder = nn.Sequential(
        	nn.Conv2d(128, 256, 5),
            nn.ReLU(inplace=True),
            nn.Dropout2d(),
            nn.Conv2d(256, 256, 1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(),
            nn.Conv2d(256, self.n_class, 1)
        	)


    def forward(self, x):
        temp = self.encoder(x)
        output = self.decoder(temp)
        output = F.interpolate(output, (256,256), mode='bicubic')
        return output

In [10]:
def cross_entropy2d(input, target, weight=None, size_average=True):
    # input: (n, c, h, w), target: (n, h, w) or might be (n, c, h, w) but c1 != c2 idk
    n, c, h, w = input.size()
    # log_p: (n, c, h, w)
    log_p = F.log_softmax(input, dim=1)
    # log_p: (n*h*w, c)
    log_p = log_p.transpose(1, 2).transpose(2, 3).contiguous()
    log_p = log_p[target.view(n, h, w, 1).repeat(1, 1, 1, c) >= 0]
    log_p = log_p.view(-1, c)
    # target: (n*h*w,)
    mask = target >= 0
    target = target[mask]
    loss = F.nll_loss(log_p, target, weight=weight, reduction='sum')
    if size_average:
        loss /= mask.data.sum()
    return loss

name = 'starter_net'
net = Net().to(device)
criterion = cross_entropy2d #nn.CrossEntropyLoss() #TODO decide loss
optimizer = torch.optim.Adam(net.parameters(), 1e-3, weight_decay=1e-5)


In [11]:
train(train_loader, net, criterion, optimizer, device, 3) #why isnt this printing loss


  0%|          | 0/80 [00:00<?, ?it/s]

torch.Size([10, 3, 256, 256])
torch.Size([10, 128, 64, 64])
torch.Size([10, 5, 60, 60])
torch.Size([10, 5, 256, 256])


  1%|▏         | 1/80 [00:10<14:04, 10.69s/it]

torch.Size([10, 3, 256, 256])
torch.Size([10, 128, 64, 64])
torch.Size([10, 5, 60, 60])
torch.Size([10, 5, 256, 256])


  2%|▎         | 2/80 [00:23<14:37, 11.24s/it]

...

 68%|██████▊   | 54/80 [15:56<06:48, 15.70s/it]

torch.Size([10, 3, 256, 256])
torch.Size([10, 128, 64, 64])
torch.Size([10, 5, 60, 60])
torch.Size([10, 5, 256, 256])


 69%|██████▉   | 55/80 [16:10<06:26, 15.46s/it]

torch.Size([10, 3, 256, 256])
torch.Size([10, 128, 64, 64])
torch.Size([10, 5, 60, 60])
torch.Size([10, 5, 256, 256])


KeyboardInterrupt: 

In [21]:
# TODO: create a OneHotEncoder object, and fit it to all of X
from sklearn import preprocessing
X_2 = np.array([1,2,3,2,1,4,3,2,1]).reshape((-1, 1))
print(X_2)
# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(X_2)

# 3. Transform
onehotlabels = enc.transform(X_2).toarray()
onehotlabels

#

[[1]
 [2]
 [3]
 [2]
 [1]
 [4]
 [3]
 [2]
 [1]]


/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])

In [29]:
np.argmax(onehotlabels, axis=1)

array([0, 1, 2, 1, 0, 3, 2, 1, 0])

In [26]:
from sklearn.metrics import average_precision_score as ap_score
ap_score(onehotlabels[:,1], onehotlabels[:,2])

0.3333333333333333